# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import gkey

gmaps.configure(api_key = gkey)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_sum_path = "../output_data/cities.csv"
city_sum = pd.read_csv(city_sum_path)
city_sum

,City Name,Longitude,Latitude,Max Temperature,Humidity,Cloud Coverage,Wind Speed,City Country,Date Time
0,Cabo San Lucas,-109.91,22.89,91.00,58,20,14.99,MX,1594675620
1,Puerto Ayora,-90.35,-0.74,70.00,90,56,1.01,EC,1594675748
2,Carnarvon,113.63,-24.87,60.80,93,75,4.70,AU,1594675748
3,Kawhia,174.82,-38.07,57.00,97,100,5.99,NZ,1594675749
4,Torbay,-52.73,47.67,77.00,78,20,17.22,CA,1594675749
...,...,...,...,...,...,...,...,...,...
537,Ballina,153.57,-28.87,50.00,66,0,12.75,AU,1594675880
538,Mporokoso,30.12,-9.37,59.41,60,0,7.45,ZM,1594675880
539,Vestmannaeyjar,-20.27,63.44,51.96,84,100,5.06,IS,1594675880
540,Pundaguitan,126.17,6.37,83.21,70,95,2.89,PH,1594675881


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = city_sum[["Latitude", "Longitude"]]
humid = city_sum["Humidity"]

In [4]:
fig = gmaps.figure()
humidity_heat_map = gmaps.heatmap_layer(locations, weights=humid,
                                        dissipating=False, max_intensity=10,
                                        point_radius=3)
fig.add_layer(humidity_heat_map)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.
* A max temperature lower than 80 degrees but higher than 70.


* Wind speed less than 10 mph.


* Zero cloudiness.


* Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

In [5]:
ideal_places = city_sum.loc[(city_sum['Max Temperature'] > 70) &
                            (city_sum['Max Temperature'] < 80) &
                            (city_sum['Wind Speed'] < 10) & 
                            (city_sum['Cloud Coverage'] == 0)]
                            
ideal_places[:10]

,City Name,Longitude,Latitude,Max Temperature,Humidity,Cloud Coverage,Wind Speed,City Country,Date Time
70,Mahon,4.27,39.89,73.40,78,0,9.17,ES,1594675764
151,Acarí,-74.62,-15.43,73.47,43,0,6.69,PE,1594675784
162,Paraopeba,-44.40,-19.28,78.80,47,0,8.05,BR,1594675787
167,Fasa,53.65,28.94,73.40,20,0,3.62,IR,1594675654
318,Ürümqi,87.60,43.80,71.60,43,0,8.95,CN,1594675828
342,Kumul,93.45,42.80,71.83,38,0,8.55,CN,1594675834
356,Herat Province,62.00,34.50,72.99,30,0,8.55,AF,1594675837
384,Erenhot,111.98,43.65,70.38,52,0,6.53,CN,1594675843
426,Sidi Ali,0.42,36.10,71.65,80,0,8.19,DZ,1594675854
483,Grand-Santi,-54.38,4.25,75.61,91,0,1.16,GF,1594675868


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_places[:10].copy()
hotel_df['Hotel Name'] = ""
hotel_df.head()

,City Name,Longitude,Latitude,Max Temperature,Humidity,Cloud Coverage,Wind Speed,City Country,Date Time,Hotel Name
70,Mahon,4.27,39.89,73.40,78,0,9.17,ES,1594675764,
151,Acarí,-74.62,-15.43,73.47,43,0,6.69,PE,1594675784,
162,Paraopeba,-44.40,-19.28,78.80,47,0,8.05,BR,1594675787,
167,Fasa,53.65,28.94,73.40,20,0,3.62,IR,1594675654,
318,Ürümqi,87.60,43.80,71.60,43,0,8.95,CN,1594675828,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotel_radius = 5000
params = {
    "radius": hotel_radius,
    "type": "lodging",
    "keyword": "hotel", 
    "key": gkey}

for index, row in hotel_df.iterrows():
    hotel_lat = row['Latitude']
    hotel_lng = row['Longitude']
    params['location'] = f"{hotel_lat}, {hotel_lng}"
    hotel_resp = requests.get(base_url, params=params).json()
#     pprint(hotel_resp)
    results = hotel_resp['results']
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    except (KeyError, IndexError):
        hotel_df.loc[index, 'Hotel Name'] = "Not Available"
        pass

In [8]:
hotel_df

,City Name,Longitude,Latitude,Max Temperature,Humidity,Cloud Coverage,Wind Speed,City Country,Date Time,Hotel Name
70,Mahon,4.27,39.89,73.40,78,0,9.17,ES,1594675764,Casa Ládico Boutique Hotel
151,Acarí,-74.62,-15.43,73.47,43,0,6.69,PE,1594675784,Hostal Mi Refugio
162,Paraopeba,-44.40,-19.28,78.80,47,0,8.05,BR,1594675787,HC Hotel Paraopeba
167,Fasa,53.65,28.94,73.40,20,0,3.62,IR,1594675654,Jahangardi Hotel
318,Ürümqi,87.60,43.80,71.60,43,0,8.95,CN,1594675828,JinJiang International Hotel Urumqi
342,Kumul,93.45,42.80,71.83,38,0,8.55,CN,1594675834,7 Days Premium
356,Herat Province,62.00,34.50,72.99,30,0,8.55,AF,1594675837,Not Available
384,Erenhot,111.98,43.65,70.38,52,0,6.53,CN,1594675843,Moli International Hotel
426,Sidi Ali,0.42,36.10,71.65,80,0,8.19,DZ,1594675854,Not Available
483,Grand-Santi,-54.38,4.25,75.61,91,0,1.16,GF,1594675868,Hotel La Villa Couacou


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{City Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations,
                            info_box_content=hotel_info)
fig.add_layer(markers)
fig.add_layer(humidity_heat_map)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))